Notebook for preparing and testing lightning data and trainer modules to be used.

First, the data module, which will be pretty easy as I made the loaders in a different spot. 

In [1]:
import sys
sys.path.append('../src')
from dataloaders import create_dataloaders
import matplotlib.pyplot as plt
%matplotlib inline

train_loader, _, _ = create_dataloaders(prior_years=3,crypto='bitcoin',values='usd',batch_size=8,buy_thresh=3,labels_to_load=['pct_change'],window=7)

for batch in train_loader:
    break

TypeError: create_dataloaders() missing 2 required positional arguments: 'prior_days' and 'pct_window'

In [5]:
import pytorch_lightning as pl

In [8]:
class CryptoDataModule(pl.LightningDataModule):
    def __init__(self, 
                 crypto: str = "bitcoin", 
                 prior_years: int = 5,
                 values: str = 'usd',
                 buy_thresh: int = 3,
                 labels_to_load: list = ['pct_change'],
                 window: int = 14,
                 batch_size: int = 32):
        super().__init__()
        self.crypto = crypto
        self.prior_years = prior_years
        self.values = values
        self.buy_thresh = buy_thresh
        self.labels_to_load = labels_to_load.split(',')
        self.window = window
        self.batch_size = batch_size

    def setup(self):
        self.train_loader, self.val_loader, self.test_loader = create_dataloaders(prior_years=self.prior_years,
                                                                                  crypto=self.crypto,
                                                                                  values=self.values,
                                                                                  batch_size=self.batch_size,
                                                                                  buy_thresh=self.buy_thresh,
                                                                                  labels_to_load=self.labels_to_load,
                                                                                  window=self.window)


    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.val_loader

    def test_dataloader(self):
        return self.test_loader

 

In [1]:
class CryptoTrainer(pl.LightningModule):
    """
    """
    def __init__(self,args,
                 n_layers,
                 filter_size,
                 dropout_rate):
        super().__init__()

        self.save_hyperparameters()
    
    
        hidden_size = args.window_size
    
        self.model =  Transformer(n_layers=n_layers,
                 hidden_size=hidden_size,
                 filter_size=filter_size,
                 dropout_rate=dropout_rate,
                window_size=args.window_size)
        
        # todo - some wandb logging
        

    def forward(self,x):

        return self.model(x)

    def training_step(self,batch, batch_idx):
        X, y = batch

        y_pred = self(X)

        loss = F.mse_loss(y,y_pred)

        self.log('train_loss', loss, on_epoch=True)
        # TODO , on every X steps maybe examine the plot and see what I think?
        return loss

    # validation step
    def validation_step(self,batch,batch_idx):
        X, y = batch

        loss = F.mse_loss(y,y_pred)

        self.log('valid_loss', loss, on_epoch=True)

        return loss


    #test step
    def test_step(self,batch,batch_idx):
        X, y = batch

        loss = F.mse_loss(y,y_pred)

        self.log('test_loss', loss, on_epoch=True)

        return loss


    # optimizers

    def configure_optimizers(self):
        optimizer =  torch.optim.Adam(self.parameters(), lr = self.learning_rate ,weight_decay = self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience = 4)
        return [optimizer], [scheduler] 

        
        # args

    @staticmethod
    def add_model_specific_args(parent_parser):

        parser = ArgumentParser(parents=[parent_parser], add_help=False)

        parser.add_argument('--learning_rate', type=float, default=3e-4)
        parser.add_argument('--n_layers', type=int,default=12) 
        parser.add_argument('--filter_size', type=int,default=12) 
        parser.add_argument('--weight_decay', type=float, default=3e-4)
        parser.add_argument('--dropout_rate', type=float, default=.1)

        
        return parser

        
        


NameError: name 'pl' is not defined

Shopping list of arguments to get

batch size

model specific ones


window size

prior years

crypto
value
threshold 
labels to load 

Also need to ensure how variables are declared. Args or args.?

In [ ]:
def main():

    pl.seed_everything(42)

    parser = ArgumentParser()

    parser.add_argument('--batch_size', type=int,
                        default=32)
    parser.add_argument('--prior_years', type=int,
                        default=5)
    parser.add_argument('--window_size', type=int,
                        default=28)
    parser.add_argument('--crypto', type=str,
                        default="bitcoin")
    parser.add_argument('--values', type=str,
                        default='usd')
    parser.add_argument('--labels_to_load', type=str,
                        default='pct_change')

    parser = pl.Trainer.add_argparse_args(parser)
    parser = CryptoTrainer.add_model_specific_args(parser)
    args = parser.parse_args()

    # ------------
    # training
    # ------------


    checkpoint_callback = ModelCheckpoint(
        save_last = True,
    save_top_k=1,
    verbose=True,
    monitor="valid_loss",
    mode="min"
    )

    early_stopping_callback = EarlyStopping(
                       monitor='valid_loss',
                       min_delta=0.00,
                       patience=30,
                       verbose=False,
                       mode='min'
                    )

    lr_monitor = LearningRateMonitor(logging_interval= 'step')

    run = wandb.init()#entity="automated-reporting-fdl2021",project="ESTR")


    wandb_logger = WandbLogger() #where to specify TODO


    trainer = pl.Trainer().from_argparse_args(args,logger=wandb_logger,
                       callbacks=[checkpoint_callback,lr_monitor,early_stopping_callback])



    # ------------
    # data
    # ------------
    data_module = CryptoDataModule(crypto=args.crypto,
                 prior_years=args.prior_years,
                 values=args.values,
                 buy_thresh,=args.buy_thresh
                 labels_to_load=args.labels_to_load,
                 window=args.window_size,
                 batch_size=args.batch_size,
            )


    # ------------
    # model
    # ------------

    trader = CryptoTrainer(args,
                 n_layers=args.n_layers,
                 filter_size=args.filter_size,
                 dropout_rate=args.dropout_rate)
        
    
    
  #  ESTR(args,
    #    model_name_or_path=args.model_name_or_path,
     #   n_layers=args.n_layers,
        learning_rate=args.learning_rate,
        adam_beta1=args.adam_beta1,
        adam_beta2=args.adam_beta2,
        adam_epsilon=args.adam_epsilon,
        weight_decay=args.weight_decay,
        load_pretrained = args.load_pretrained,
        num_warmup_steps = num_warmup_steps,
        num_training_steps = num_training_steps,
        num_cycles = num_cycles,
                #    )


    if args.auto_lr_find:
        trainer.tune(trader, data_module)



    trainer.fit(trader, data_module)


    run.log({"training_samples" : trader.train_text_table})
    run.log({"valid_samples" : trader.valid_text_table})
    run.log({"test_samples" : trader.test_text_table})


    wandb.finish()


if __name__ == '__main__':

    main()

    

In [11]:
datamodule = CryptoDataModule()
datamodule.setup()

In [16]:
for X, y in datamodule.train_dataloader():
    break

In [17]:
X.shape

torch.Size([32, 14])

Now the trainer.

In [18]:
from model.btc_transformer import Transformer

In [19]:
model = Transformer(hidden_size=14,window_size=14)

In [23]:
out = model(X)

torch.Size([32, 14])
torch.Size([32, 14, 14])


In [27]:
from torch.nn.functional import mse_loss

Now the CryptoTrainer



In [29]:
loss = mse_loss(out,y)

In [30]:
loss

tensor(135.8867, grad_fn=<MseLossBackward>)